In [1]:
# iimport all the necessary packages
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
import pandas as pd

In [2]:
# import os and find the path
import os
print(os.getcwd())

/Users/alexandros.samartzis/Spiced_Academy/stationary-sriracha-student-code/Week_2/spiced_projects


In [3]:
# load data
titanik = pd.read_csv('train.csv', index_col=0)


In [4]:
# Delete unnecessary columns
del titanik['Ticket']
del titanik['Name']
#del titanik['Embarked']
del titanik['Cabin']

In [5]:
# Find in a table the NaN of each Column
titanik.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
# Find the unique values of the column
titanik['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [7]:
# split to X, and y
X = titanik.iloc[:,1:]
y = titanik['Survived']

In [8]:
X.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22.0,1,0,7.2500,S
2,1,female,38.0,1,0,71.2833,C


In [9]:
# create a sequential pipeline 
# output of one step will be input to the next 
numeric_transformer_OHE = make_pipeline(
    OneHotEncoder(sparse=False, handle_unknown='ignore')
    )

In [10]:
# create a sequential pipeline 
# output of one step will be input to the next 
numeric_transformer_BIN = make_pipeline(
    KBinsDiscretizer(n_bins=5, encode='onehot', strategy='quantile')
    )

In [11]:
# create a sequential pipeline 
# output of one step will be input to the next 
numeric_transformer_SCALE = make_pipeline(
    StandardScaler()
    )

In [12]:
# reate custom functions

def name_length(df):
    length = df[df.columns[0]].str.len()
    return length.values.reshape(-1, 1)

In [13]:
# feature engineering for categorical
categorical_features = ["Embarked","Sex", "Pclass"]

In [14]:
# handle unkno
categorical_transformer_OHE = OneHotEncoder(handle_unknown="ignore")

In [15]:
# create a sequential pipeline 
# output of one step will be input to the next 
categorical_transformer_BIN = make_pipeline(
    KBinsDiscretizer(n_bins=5, encode='onehot', strategy='quantile')
    )

In [16]:
# create a sequential pipeline for getting rid of the NaN for numerical
numeric_imputer = make_pipeline(
    SimpleImputer(strategy='mean')
    )

In [17]:
# create a sequential pipeline for getting rid of the NaN for numerical
categorical_imputer = make_pipeline(
    SimpleImputer(strategy='most_frequent')
    )

In [18]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num_imputer", numeric_imputer, ['Age']),
        ("num_OHE", numeric_transformer_OHE, ["Pclass", 'SibSp']),
        ("num_AGE", numeric_transformer_BIN, ["Age"]),
        ("num_SCALE", numeric_transformer_SCALE, ["Age","Parch","Fare"]),
        ("cat_imputer", categorical_imputer, ['Embarked']),
        ("cat", categorical_transformer_OHE, categorical_features)
    ],
    remainder='passthrough')

In [19]:
# create the model pipeline
pipeline = make_pipeline(preprocessor, LogisticRegression(max_iter=300))


In [20]:
# split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)

In [21]:
X.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [22]:
# fit the pipeline to training data
pipeline.fit_transform(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# calculate the accuracy score from test data
print("model score: %.3f" % pipeline.score(X_test, y_test))

In [ ]:
# get predictions from the pipeline
print(pipeline.predict(X_test))

In [ ]:
# get prediction probabilities from the pipeline 
print(pipeline.predict_proba(X_test)[:, 1])